Notes:

1. We use the environment `topsapp_env` described in [environment.yml](https://github.com/ACCESS-Cloud-Based-InSAR/DockerizedTopsApp/blob/dev/environment.yml) in the `DockerizedTopsApp` repo. 
2. You will have to launch this notebook from a terminal in which the environment is activated since this notebook sends commands to the terminal (specifically `isce2` commands)

The code below is based on this [file](https://github.com/ACCESS-Cloud-Based-InSAR/DockerizedTopsApp/blob/dev/isce2_topsapp/localize_dem.py) which stages our DEMs for the production of ARIA GUNWs.

*Warning*: the environment is extraordinarily finnicky with rasterio, gdal and we had to enforce the use of slightly older versions of gdal.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dem_stitcher.stitcher import stitch_dem
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
import site
from lxml import etree
import isce
from shapely.geometry import box
import numpy as np

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


# Parameters

In [3]:
dem_name = 'glo_30'
bounds = [-169.0, 53., -167.0, 54.]

In [4]:
def tag_dem_xml_as_ellipsoidal(dem_path: Path) -> str:
    xml_path = str(dem_path) + '.xml'
    assert(Path(xml_path).exists())
    tree = etree.parse(xml_path)
    root = tree.getroot()

    y = etree.Element("property", name='reference')
    etree.SubElement(y, "value").text = "WGS84"
    etree.SubElement(y, "doc").text = "Geodetic datum"

    root.insert(0, y)
    with open(xml_path, 'wb') as file:
        file.write(etree.tostring(root, pretty_print=True))
    return xml_path


def fix_image_xml(isce_raster_path: str) -> str:
    isce_apps_path = site.getsitepackages()[0] + '/isce/applications'
    fix_cmd = [f'{isce_apps_path}/fixImageXml.py',
               '-i',
               str(isce_raster_path),
               '--full']
    fix_cmd_line = ' '.join(fix_cmd)
    subprocess.check_call(fix_cmd_line, shell=True)
    print(fix_cmd_line)
    return isce_raster_path


def download_dem_for_isce2(extent: list,
                           dem_name: str = 'glo_30',
                           dem_dir: Path = None,
                           buffer: float = .004) -> dict:
    """
    Parameters
    ----------
    extent : list
        [xmin, ymin, xmax, ymin] for epsg:4326 (i.e. (x, y) = (lon, lat))
    dem_name : str, optional
        See names in `dem_stitcher`
    dem_dir: Path, optional
        
    buffer : float, optional
        In degrees, by default .001, which is .5 km at equator
    Returns
    -------
    Path
    """
    dem_dir = dem_dir or Path(f'{dem_name}')
    dem_dir.mkdir(exist_ok=True, parents=True)

    extent_geo = box(*extent)
    extent_buffered = list(extent_geo.buffer(buffer).bounds)
    extent_buffered = [np.floor(extent_buffered[0]), np.floor(extent_buffered[1]),
                       np.ceil(extent_buffered[2]), np.ceil(extent_buffered[3])]

    # you can remove this parameter if you don't mind resolution of original DEM
    dem_res = 0.0002777777777777777775
    dem_array, dem_profile = stitch_dem(extent_buffered,
                                        dem_name,
                                        dst_ellipsoidal_height=True,
                                        dst_area_or_point='Point',
                                        n_threads_downloading=5,
                                        # ensures square resolution
                                        dst_resolution=dem_res
                                        )

    dem_path = dem_dir / 'full_res.dem.wgs84'
    dem_array[np.isnan(dem_array)] = 0.

    dem_profile_isce = dem_profile.copy()
    dem_profile_isce['nodata'] = None
    dem_profile_isce['driver'] = 'ISCE'
    # remove keys that do not work with ISCE gdal format
    [dem_profile_isce.pop(key) for key in ['blockxsize', 'blockysize', 'compress', 'interleave', 'tiled']]

    with rasterio.open(dem_path, 'w', **dem_profile_isce) as ds:
        ds.write(dem_array, 1)
        
    dem_xml = tag_dem_xml_as_ellipsoidal(dem_path)
    fix_image_xml(dem_xml)

    return dem_xml

In [5]:
%%time

download_dem_for_isce2(extent=bounds,
                       dem_name=dem_name,
                       dem_dir=Path('isce_dem'))

Reading glo_30 Datasets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.44it/s]


This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /Users/cmarshak/bekaert-team/dem-stitcher/notebooks/isce_dem/full_res.dem.wgs84
/Users/cmarshak/opt/anaconda3/envs/dem-stitcher-isce/lib/python3.9/site-packages/isce/applications/fixImageXml.py -i isce_dem/full_res.dem.wgs84.xml --full
CPU times: user 21.8 s, sys: 5 s, total: 26.8 s
Wall time: 46.4 s


'isce_dem/full_res.dem.wgs84.xml'